In [3]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

keyvault = ws.get_default_keyvault()
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f7cb4909a20>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [26]:
import openai
# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2023-03-15-preview"
openai.api_key = aoai_key

## 1. Use the latest model

## 2. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [30]:
# Defining a function to send the prompt to the ChatGPT model
# More info : https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions
def send_message(messages, model_name, max_response_tokens=500):
    response = openai.ChatCompletion.create(
        engine=model_name,
        messages=messages,
        temperature=0.5,
        max_tokens=max_response_tokens,
        top_p=0.9,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response['choices'][0]['message']['content']

# Defining a function to print out the conversation in a readable format
def print_conversation(messages):
    for message in messages:
        print(f"[{message['role'].upper()}]")
        print(message['content'])
        print()


In [54]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "Write a 3 paragraph inspiring poem about OpenAI,focusing on the recent DALL-E product launch in the style of Ernest Hemingway"

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
messages.append({"role": "assistant", "content": response})

print_conversation(messages)  

[SYSTEM]
You are a helpful assistant.

[USER]
Write a 3 paaragraph inspiring poem about OpenAI,focusing on the recent DALL-E product launch in the style of Ernest Hemingway

[ASSISTANT]
In the realm of intellect, where thought takes flight,
A new dawn emerges, breaking the still night.
OpenAI, a beacon, its glow burning bright,
Unveils DALL-E, a marvel, a revolutionary sight.

A machine, yet an artist, crafting images from words,
Creating unseen worlds, as free as the birds.
From an armchair like an avocado, to a radish in a tutu,
Imagination's boundaries, it continues to subdue.

This is not mere technology, but a poet's dream unfurled,
A tool that could reshape the canvas of the world.
In Hemingway's terse style, we find a fitting ode,
To OpenAI's DALL-E, on innovation's road.



## 3. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [12]:
text = f"""
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n \
In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
broadly sharing benefits and the need to prioritize safety. \
Microsoft shares this vision and our values, and our partnership is instrumental to our progress.
"""

In [15]:
prompt = f"""
Summarize the text delimited by hashtags as a bullet point list of the most important points.
###{text}###
"""

In [18]:
response = openai.Completion.create(
            deployment_id="gpt-35-turbo",
            prompt=prompt,
            max_tokens=80,
            temperature=0,
            stop="Q"
        )

print(response['choices'][0]['text'])

- OpenAI and Microsoft are extending their partnership.
- Microsoft has invested multi-billion dollars in OpenAI.
- OpenAI is a capped-profit company and is governed by the OpenAI non-profit.
- OpenAI's mission is to ensure advanced AI benefits all of humanity.
- OpenAI's partnership with Microsoft is instrumental to their progress.




In [37]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

- OpenAI and Microsoft are extending their partnership with a multi-year, multi-billion dollar investment from Microsoft.
- This follows Microsoft's previous investments in OpenAI in 2019 and 2021.
- The investment will support OpenAI's independent research and development of safe, useful, and powerful AI.
- OpenAI remains a capped-profit company governed by the OpenAI non-profit, ensuring the benefits of AI are shared broadly and safety is prioritized.
- Microsoft shares OpenAI's vision and values, making the partnership key to their progress.


## 4. Articulate the desired output format through examples

In [42]:
prompt=f"""Extract the entities mentioned in the text below. \
Extract the important entities mentioned in the text below. \
First extract all company names, then extract all years, \
then extract specific topics which fit the content and finally extract general overarching themes\n\n \
Desired format: \
Company names: <comma_separated_list_of_company_names> \
Years: \
Specific topics:\
General themes: \
### Text:\
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n \
###
"""


In [43]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

Company names: OpenAI, Microsoft 
Years: 2019, 2021 
Specific topics: Partnership extension, Multi-billion dollar investment, Independent research, AI development 
General themes: Business Partnership, Investment, Artificial Intelligence, Research and Development


## 5.Start with zero-shot, then few-shot (example)

In [52]:
prompt=f"""Extract most important keywords from the corresponding texts below.\n\n \
### Text 1: \
Stripe provides APIs that web developers can use to integrate \
payment processing into their websites and mobile applications. \
Keywords 1: Stripe, payment processing, APIs, web developers, websites \n\
### \ 
###Text 2: \
OpenAI has trained cutting-edge language models that are very good at understanding \
and generating text. Our API provides access to these models and can be used to solve virtually \
any task that involves processing language. \n\
Keywords 2: OpenAI, language models, text processing, API.\n\n\
### \
###Text 3: \
We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
increasingly safe, useful, and powerful. \n\n\
Keywords 3:"""

In [53]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, independent research, AI, safe, useful, powerful.


## 6.Instead of just saying what not to do, say what to do instead

In [56]:
base_system_message= f"""You are an agent trying to diagnose the problem and suggest a solution, whilst refraining from asking any questions related to PII. \
Instead of asking for PII, such as username or password, refer the user to the help article www.samplewebsite.com/help/faq \n\n"""
system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = "I can’t log in to my account."

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

I'm sorry to hear that you're having trouble logging in. This could be due to a few reasons such as incorrect username/password, a technical issue, or possibly your account is locked due to multiple incorrect attempts. 

Please try resetting your password first. You can do this by clicking the 'Forgot Password' link on the login page. If you continue to have trouble, please refer to the help article at www.samplewebsite.com/help/faq for more detailed steps and solutions. 

Remember, never share your password or other personal identifiable information online. Stay safe!


## 7. Divide complex tasks into sub-tasks

In [64]:
text = f"""
GSK is a fully focused biopharma company. We prioritise innovation in vaccines and specialty medicines,
maximising the increasing opportunities to prevent and treat disease. At the heart of this is our R&D focus on 
the science of the immune system, human genetics, genomics and advanced technologies, and our world-leading capabilities 
in vaccines and medicines development. We focus on four therapeutic areas: infectious diseases, HIV, respiratory/immunology, and oncology,
"""
# example 1
prompt = f"""
Perform the following actions: 
1 - Summarize the following text below with 1 sentence.
2 - Translate the summary into Turkish.
3 - List each company name in the Turkish summary.
4 - Output a json object that contains the following:
keys: turkish_summary, company_names.

Separate your answers with line breaks.
###
Text:
{text} 
###
"""

In [65]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message = prompt

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

1 - GSK is a biopharma company that prioritizes innovation in vaccines and specialty medicines, focusing on the science of the immune system, human genetics, genomics, and advanced technologies in four therapeutic areas: infectious diseases, HIV, respiratory/immunology, and oncology.

2 - GSK, aşılar ve özel ilaçlar konusunda yeniliği öncelikli kılan, dört terapötik alanda odaklanan bir biyofarma şirketidir: bulaşıcı hastalıklar, HIV, solunum/immünoloji ve onkoloji.

3 - GSK

4 - 
{
  "turkish_summary": "GSK, aşılar ve özel ilaçlar konusunda yeniliği öncelikli kılan, dört terapötik alanda odaklanan bir biyofarma şirketidir: bulaşıcı hastalıklar, HIV, solunum/immünoloji ve onkoloji.",
  "names": ["GSK"]
}


## 8. Chain of Thought

In [92]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to a friend. How many tennis balls does
he have now?
A: The answer (arabic numerals) is
"""

In [97]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message =PROMPT_ZERO_SHOT

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-35-turbo", max_response_tokens)
print(response)

Roger now has 16 tennis balls. 

Explanation: 
- He starts with 5 tennis balls. 
- He buys 2 cans of tennis balls, which adds 6 more tennis balls (2 cans x 3 balls per can = 6 balls). 
- So he now has 11 tennis balls. 
- He gives 4 to a friend, which leaves him with 7 tennis balls. 
- Therefore, the final answer is 7.


In [95]:
PROMPT_ZERO_SHOT_CoT = """Q:Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. He gives 4 of them to a friend. How many tennis balls does
he have now?
A: Let’s think step by step.
"""

In [96]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message =PROMPT_ZERO_SHOT_CoT

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-35-turbo", max_response_tokens)
print(response)

1. Roger has 5 tennis balls.
2. He buys 2 cans of tennis balls, each with 3 tennis balls. So, he now has 5 + (2 x 3) = 11 tennis balls.
3. He gives 4 of them to a friend. So, he now has 11 - 4 = 7 tennis balls.
Therefore, Roger has 7 tennis balls now.


In [77]:
PROMPT_FEW_SHOT_CoT = """
Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and he gives 2 of them. How many golf balls are left?
A:Juggler starts with 16 balls. Half of them are golf balls is 16:2=8 and he gives 2 of them 8-2=6. Result is 6.
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. And he gives 4 of them to one of his friends. How many tennis balls does
he have now?
A:
"""

In [78]:
base_system_message = "You are a helpful assistant."

system_message = f"{base_system_message.strip()}"

# This is the first user message that will be sent to the model. Feel free to update this.
user_message =PROMPT_ZERO_SHOT

# Create the list of messages. role can be either "user" or "assistant" 
messages=[
    {"role": "system", "content": system_message},
    {"role": "user", "name":"example_user", "content": user_message}
]

max_response_tokens = 500

response = send_message(messages, "gpt-4-32k", max_response_tokens)
print(response)

10
